# 6 Day Turnarounds and Preparation day advantage - 6 Nations Rugby

I wanted to check whether 6-Day turn-arounds made teams more vulnerable to a defeat in Rugby Union. 

These data were originally extracted from ESPN at the link in the wiki. Their data is provided per team, which provides two rows per match, so re-organised it to have one entry per match, with the data from the Home team's perspective in each case. This provides more data per match and importantly highlights differences in prep time, which I thought might be more important than 6 day turnarounds per se.

So, let's import our data and have a quick look at what we have.

In [1]:
data = read.csv("sixnations_6dayto.csv")
# data is set up with one row per game  - Team is home team

## let's take out greater than 1 day difference in prep (4 games)
data = data[data$diff.Date.adv<2 & data$diff.Date.adv> -2,]

## Initial coarse look
writeLines("Total Home wins, losses and draws")
aggregate(GMD~Result, data, length)

writeLines("")


writeLines("Average Home win rate")
round(mean(data$Result=="won"),2)

Total Home wins, losses and draws


Result,GMD
draw,5
lost,83
won,136



Average Home win rate


[1] 0.61

So, there is a solid advantage to being the home team in Six Nations rugby. No surprise there.

Now, let's check the Home win rate for 6 day turn arounds by the Home and Away teams

In [2]:
writeLines("Home win rate when Home have 6 day TOs? Higher than avg")
with(data[data$diff.Date<7,], mean(Result=="won"))
writeLines("Home win rate when Opp have a 6 day TOs? Highest")
with(data[data$diff.Date.Opp<7,], mean(Result=="won"))

writeLines("Average Home points diff")
round(mean(data$Diff),2) # 

writeLines("Average Home points diff in when home team has 6 day TO? Below but close to avg")
with(data[data$diff.Date<7,], round(mean(Diff),2))
writeLines("Average Home points diff  when Opp have a 6 day TOs? Higher than average")
with(data[data$diff.Date.Opp<7,], round(mean(Diff),2))

Home win rate when Home have 6 day TOs? Higher than avg


[1] 0.68

Home win rate when Opp have a 6 day TOs? Highest


[1] 0.75

Average Home points diff


[1] 3.74

Average Home points diff in when home team has 6 day TO? Below but close to avg


[1] 3.2

Average Home points diff  when Opp have a 6 day TOs? Higher than average


[1] 4.62

So, if you're the home team, you are more likely to win if either you have 6 dat turnaround or the opposition. Why? 

Points adv increases if the opposition have a 6 day TO.

Let's check whether the matches in each category (6 day TO Home and 6 Day TO Away) are as easy as the average.

We'll use the Win Rate for that year as an index of team strength and subtract Away from Home to get the Home team strength advantage. This is problematic as the Win Rate includes the games we are predicting, but it'll do as a first approximation.

In [3]:
data$Year.WR.diff = data$Year.WR - data$Year.WR.Opp
# hist(data$Year.WR.diff) if needed
writeLines("Average Home Win Rate adv - is the Home team stronger on average? No") 
round(mean(data$Year.WR.diff),2)
writeLines("Home adv in Win Rate when Home have 6 day TOs? Slight")
with(data[data$diff.Date<7,], round(mean(Year.WR.diff),2))
writeLines("Home adv in Win Rate when Opp have 6 day TOs? Highest")
with(data[data$diff.Date.Opp<7,], round(mean(Year.WR.diff),2))
writeLines("Some of the effect of 6 day turnarounds is due to weaker teams as opposition for these matches")


Average Home Win Rate adv - is the Home team stronger on average? No


[1] 0

Home adv in Win Rate when Home have 6 day TOs? Slight


[1] 0.01

Home adv in Win Rate when Opp have 6 day TOs? Highest


[1] 0.03

Some of the effect of 6 day turnarounds is due to weaker teams as opposition for these matches


### Consider differences in prep time 
6/13 days for home vs 7/14 for opp (-1)

7/14 days for home vs 7/14 for opp (0)

7/14 days for home vs 6/13 for opp (+1)

#### Does prep time difference (diff.Date.avd) affect the probability of winning a six nations game?

In [9]:
round(aggregate(Result=="won"~diff.Date.adv, data, mean),2)

diff.Date.adv,"Result == ""won"""
-1,0.59
0,0.55
1,0.76


Yes.

#### Are there differences in team strength (Year.WR.diff) that align with prep time difference (diff.Date.avd)?

In [8]:
round(aggregate(Year.WR.diff~diff.Date.adv, data, mean),2)

diff.Date.adv,Year.WR.diff
-1,0.03
0,-0.05
1,0.07


This is tricky to analyse. Just looking at the date differences, there seems to be an effect. In particular, the home team has an advantage if the away team has one less day of prep.

But, this is complicated by the fact that it seems that the opposition teams that have one less day of prep tend to be weaker teams based on their overall performance that year (but it is hard to exclude the effect of the trun-around loss on their overall performance).

### Predicting wins based on differences in prep time

In [19]:
data$diff.Date.fac = as.factor(data$diff.Date.adv)
contrasts(data$diff.Date.fac) = matrix(c(1,0,0,0,0,1), ncol = 2)#-.5 # set up no diff as reference level
colnames(contrasts(data$diff.Date.fac) ) = c("One Day Less", "One Day More")
contrasts(data$diff.Date.fac)

win.diffdate.glm = glm(Result=="won" ~ diff.Date.fac, data, 
                  family = binomial(link = "logit"))
summary(win.diffdate.glm)
writeLines("Odds ratios") 
round(exp(cbind(OR = coef(win.diffdate.glm), confint(win.diffdate.glm))),3) # odds ratios 

,One Day Less,One Day More
-1,1,0
0,0,0
1,0,1



Call:
glm(formula = Result == "won" ~ diff.Date.fac, family = binomial(link = "logit"), 
    data = data)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.6775  -1.2629   0.7495   1.0942   1.0942  

Coefficients:
                          Estimate Std. Error z value Pr(>|z|)  
(Intercept)                 0.1989     0.1908   1.042   0.2972  
diff.Date.facOne Day Less   0.1806     0.3181   0.568   0.5701  
diff.Date.facOne Day More   0.9272     0.3831   2.420   0.0155 *
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 300.16  on 223  degrees of freedom
Residual deviance: 293.80  on 221  degrees of freedom
AIC: 299.8

Number of Fisher Scoring iterations: 4


Odds ratios


Waiting for profiling to be done...


,OR,2.5 %,97.5 %
(Intercept),1.220,0.840,1.779
diff.Date.facOne Day Less,1.198,0.644,2.248
diff.Date.facOne Day More,2.527,1.219,5.529


Let's control for the Home team. Some teams are stronger than others.

In [20]:
data$Team = as.factor(data$Team)

win.diffdate.glm = glm(Result=="won" ~ Team + diff.Date.fac, data, 
                  family = binomial(link = "logit"))
summary(win.diffdate.glm)
writeLines("Odds ratios") 
round(exp(cbind(OR = coef(win.diffdate.glm), confint(win.diffdate.glm))),3) # odds ratios 



Call:
glm(formula = Result == "won" ~ Team + diff.Date.fac, family = binomial(link = "logit"), 
    data = data)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.1759  -0.9375   0.6305   0.7434   1.7513  

Coefficients:
                          Estimate Std. Error z value Pr(>|z|)    
(Intercept)                1.50366    0.47660   3.155 0.001605 ** 
TeamFrance                -0.35884    0.60098  -0.597 0.550449    
TeamIreland               -0.75421    0.58668  -1.286 0.198593    
TeamItaly                 -2.79424    0.58959  -4.739 2.15e-06 ***
TeamScotland              -2.09799    0.56288  -3.727 0.000194 ***
TeamWales                 -0.98089    0.57298  -1.712 0.086914 .  
diff.Date.facOne Day Less  0.06226    0.35991   0.173 0.862665    
diff.Date.facOne Day More  0.76529    0.42171   1.815 0.069566 .  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 300.16  o

Odds ratios


Waiting for profiling to be done...


,OR,2.5 %,97.5 %
(Intercept),4.498,1.885,12.586
TeamFrance,0.698,0.206,2.260
TeamIreland,0.470,0.141,1.457
TeamItaly,0.061,0.018,0.183
TeamScotland,0.123,0.038,0.352
TeamWales,0.375,0.115,1.119
diff.Date.facOne Day Less,1.064,0.525,2.165
diff.Date.facOne Day More,2.150,0.959,5.055


Using England as our reference, Italy and Scotland are significantly less likely to win at home than England. 
The other teams are also less likely to win than England on average, but not significantly so.
The advantage of 1 day of prep time is borderline significant.

Let's control for the opposition

In [21]:
data$Opposition = as.factor(data$Opposition)

win.diffdate.glm = glm(Result=="won" ~ Team + Opposition + diff.Date.fac, data, 
                  family = binomial(link = "logit"))
summary(win.diffdate.glm)
writeLines("Odds ratios") 
round(exp(cbind(OR = coef(win.diffdate.glm), confint(win.diffdate.glm))),3) # odds ratios 


Call:
glm(formula = Result == "won" ~ Team + Opposition + diff.Date.fac, 
    family = binomial(link = "logit"), data = data)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2612  -0.7430   0.2876   0.8155   2.1141  

Coefficients:
                          Estimate Std. Error z value Pr(>|z|)    
(Intercept)                 0.8425     0.6630   1.271 0.203852    
TeamFrance                 -0.2986     0.6579  -0.454 0.649871    
TeamIreland                -0.8187     0.6466  -1.266 0.205476    
TeamItaly                  -2.8194     0.6679  -4.221 2.43e-05 ***
TeamScotland               -2.0096     0.6211  -3.235 0.001215 ** 
TeamWales                  -1.0232     0.6398  -1.599 0.109759    
Oppositionv France          0.2174     0.5398   0.403 0.687182    
Oppositionv Ireland        -0.3016     0.5395  -0.559 0.576201    
Oppositionv Italy           2.3962     0.7348   3.261 0.001110 ** 
Oppositionv Scotland        2.4519     0.7239   3.387 0.000706 ***
Opposit

Odds ratios


Waiting for profiling to be done...


,OR,2.5 %,97.5 %
(Intercept),2.322,0.648,8.921
TeamFrance,0.742,0.197,2.684
TeamIreland,0.441,0.118,1.535
TeamItaly,0.060,0.015,0.206
TeamScotland,0.134,0.037,0.431
TeamWales,0.359,0.097,1.227
Oppositionv France,1.243,0.431,3.613
Oppositionv Ireland,0.740,0.254,2.130
Oppositionv Italy,10.982,2.907,55.602
Oppositionv Scotland,11.611,3.087,54.825


Using England as our reference, home teams are significantly more likely to win if Italy and Scotland are the visiting team. Only Ireland are a more difficult opposition than England and not significantly so.

The advantage of 1 day of prep time is borderline significant.

So, I think we can say that an advantage of one day's prep time does help the Home team in the 6 Nations. 

This effect is clouded by the fact that differences in prep time tend to occur when the Home team is stronger that year (though strength is influenced by the win). 

Interestingly, when you're the home team, having one day's less prep time doesn't have a significant effect.